In [1]:
import pandas as pd
import requests
import time
import tqdm
import io

In [10]:
# appelle du département par code postal
api = 'eyJ4NXQiOiJZV0kxTTJZNE1qWTNOemsyTkRZeU5XTTRPV014TXpjek1UVmhNbU14T1RSa09ETXlOVEE0Tnc9PSIsImtpZCI6ImdhdGV3YXlfY2VydGlmaWNhdGVfYWxpYXMiLCJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiJnZGxlZHMzMUBjYXJib24uc3VwZXIiLCJhcHBsaWNhdGlvbiI6eyJvd25lciI6ImdkbGVkczMxIiwidGllclF1b3RhVHlwZSI6bnVsbCwidGllciI6IlVubGltaXRlZCIsIm5hbWUiOiJEZWZhdWx0QXBwbGljYXRpb24iLCJpZCI6MjgxOTcsInV1aWQiOiJjMDA0ZmQ1NC0xMTY3LTQ3MTEtOWQ3MC04M2ExZWI0YmI0MGYifSwiaXNzIjoiaHR0cHM6XC9cL3BvcnRhaWwtYXBpLm1ldGVvZnJhbmNlLmZyOjQ0M1wvb2F1dGgyXC90b2tlbiIsInRpZXJJbmZvIjp7IjUwUGVyTWluIjp7InRpZXJRdW90YVR5cGUiOiJyZXF1ZXN0Q291bnQiLCJncmFwaFFMTWF4Q29tcGxleGl0eSI6MCwiZ3JhcGhRTE1heERlcHRoIjowLCJzdG9wT25RdW90YVJlYWNoIjp0cnVlLCJzcGlrZUFycmVzdExpbWl0IjowLCJzcGlrZUFycmVzdFVuaXQiOiJzZWMifX0sImtleXR5cGUiOiJQUk9EVUNUSU9OIiwic3Vic2NyaWJlZEFQSXMiOlt7InN1YnNjcmliZXJUZW5hbnREb21haW4iOiJjYXJib24uc3VwZXIiLCJuYW1lIjoiRG9ubmVlc1B1YmxpcXVlc0NsaW1hdG9sb2dpZSIsImNvbnRleHQiOiJcL3B1YmxpY1wvRFBDbGltXC92MSIsInB1Ymxpc2hlciI6ImFkbWluX21mIiwidmVyc2lvbiI6InYxIiwic3Vic2NyaXB0aW9uVGllciI6IjUwUGVyTWluIn1dLCJleHAiOjE3NDg0NDI4MzEsInRva2VuX3R5cGUiOiJhcGlLZXkiLCJpYXQiOjE3NDg0MzIwMzEsImp0aSI6IjBlZWNjNmI3LTYzNDktNDYwMy04ODE2LTAyMjZmMmQyMDM0NiJ9.x2WJ94rsMIXAjfUsjSYOfAEb043sX2qX-n9GS917bS7WHNDsgRcCnXVOijauKtK6ctcIvTmAI7R7nxIvvrmhpbPwVlClm0ivR0IiQn8WbFQGBQophIqVNU5t0yuUCCPXItPOAg95aUOKF67KGvPggKLnY8E1E-OPEV-05zu3wdHLZ-ycnb9XjjuXYQn1ZYKJiqeSm7UEkErxM_j004CPRv4YLSUw-WFlPkm23JcEk_tSUWK3QNXPT6XuMKhMaJ6ukPyk5VtwA33b-NxJozQmzKPl3JCCFY5bXxiJeAzPKy7PhZ-QWN7Fsp2oO-IpBxWNWQK-mShvHsoS98jJjHTL_A=='
dep = '83'
url = 'https://public-api.meteofrance.fr/public/DPClim/v1/liste-stations/quotidienne'

params = {
    'id-departement': dep,
    'parametre': 'temperature'
}

headers = {
    'accept': '*/*',
    'apikey': api
}

corse = requests.get(url, headers=headers, params=params)

print('erreur corse', corse.status_code)
corse.json()

# on applique un mask pour ne prendre que les stations ouvertes
corse_df = pd.DataFrame(corse.json())
corse_df['posteOuvert'] = corse_df['posteOuvert'].astype(bool)
mask = corse_df['posteOuvert'] == True
corse_df = corse_df[mask]

# appelle les information par station sur plusieurs années
# all_data = pd.DataFrame()

id = corse_df['id']

for i in id:

    for années in range(2006, 2025):
        date_debut = f'{années}-01-01T00:00:00Z'
        date_fin = f'{années}-12-31T23:59:59Z'

        url = "https://public-api.meteofrance.fr/public/DPClim/v1/commande-station/quotidienne"
        params = {
            "id-station": i,
            "date-deb-periode": date_debut,
            "date-fin-periode": date_fin
        }
        headers = {
            "accept": "*/*",
            "apikey": api
        }

        corse1 = requests.get(url, headers=headers, params=params)
        print('erreur corse1', corse1.status_code)
        corse1_json = corse1.json()

        # Wrap in list if it's a dict of scalars
        corse1_df = pd.DataFrame(corse1_json).reset_index()

        name = dep + '_' + corse_df.loc[corse_df['id'] == i, 'nom'].values[0] + '_' + str(années)
       
        # Extract the 'return' value if the response is a dict
        id_cmde = corse1_df.iloc[0,1]
       

        url = "https://public-api.meteofrance.fr/public/DPClim/v1/commande/fichier"
        params = {
            "id-cmde": id_cmde
        }
        headers = {
            "accept": "*/*",
            "apikey": api
        }
        corse2 = requests.get(url, headers=headers, params=params)
        
        print('erreur corse2', corse2.status_code)
        #print(corse2.text)
        
        # output_file = name +'.csv'
        # pd.DataFrame(corse2).to_csv(output_file, index=False)

        nom_station = corse_df.loc[corse_df['id'] == i, 'nom'].values[0].replace(' ', '_')
        output_file = f"{dep}_{nom_station}_{années}.csv"

        # Enregistrement du fichier
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(corse2.text)

        time.sleep(60 / 25)  # 60 seconds divided by 25 requests
# output_file = dep + '.csv'
# all_data.to_csv(output_file, index=False)

erreur corse 200
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 204
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 204
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 204
erreur corse1 202
erreur corse2 204
erreur corse1 202
erreur corse2 204
erreur corse1 202
erreur corse2 204
erreur corse1 202
erreur corse2 204
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 204
erreur corse1 202
erreur corse2 204
erreur corse1 202
erreur corse2 204
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur cors

In [3]:
corse1_df

,index,elaboreProduitAvecDemandeResponse
0,return,2025005551724


In [4]:
id_cmde

'2025005551724'

In [5]:
output_file

'13_VAUVENARGUES_2024.csv'

In [6]:
corse_df.head(15)

,id,nom,posteOuvert,typePoste,lon,lat,alt,postePublic
5,13001009,AIX EN PROVENCE,True,1,5.424500,43.529500,173,True
9,13004003,ARLES,True,1,4.693833,43.510000,1,True
17,13005003,AUBAGNE,True,2,5.601167,43.307333,130,True
23,13022003,CASSIS,True,1,5.503167,43.222167,212,True
25,13028001,BEC DE L AIGLE,True,2,5.574167,43.174667,316,True
28,13030001,CUGES-LES-PINS,True,2,5.700167,43.274000,184,True
30,13031002,LA DESTROUSSE_SAPC,True,2,5.601000,43.386667,201,True
33,13036003,EYRAGUES,True,2,4.834833,43.847167,17,True
39,13047001,ISTRES,True,0,4.927500,43.522667,23,True
44,13054001,MARIGNANE,True,0,5.216000,43.437667,9,True


In [7]:
id_cmde

'2025005551724'